# 12. Scikit-Learn Machine Learning Pipeline

## Required steps
### Pre-process
    - [X] Categorical to Numeric
    - [X] Clean Numeric Data
    - [X] Outlier Detection
    - [X] Data Transformation
    - [X] Data Normalization
    - [X] Dimensionality Reduction
    
### Training steps - models
    - [X] Kernel Ridge
    - [X] XGBoost Regression
    - [X] Lasso Regression
    - [ ] ANN Regression
    - [X] Random Forest Regression
    - [X] ElasticNet
    - [X] Bayesian Ridge Regression
    - [X] LassoLarsIC Regression
    - [X] Gradient Boosting Regression
    
### Stacking
    - [ ] Picking one of the traning models as Meta-model
    - [ ] Use the rest for training the evaluation set and test on the test set.
    - [ ] Use Meta-model to predict the test set based on the trained models results.
    
### Ensembling [?]

---------------
## Implementation
## Pre-process classes definition

In [1]:
import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator, TransformerMixin

### Convert Categorical columns to Numeric

In [2]:
import json 
import category_encoders as ce
# categorical missing value imputer
class CatToNum(BaseEstimator, TransformerMixin):

    def __init__(self, dict_address, continuesVars, descreteVars_Ordinal, descreteVars_Nominal):
        self.continuesVars = continuesVars
        self.descreteVars_Ordinal = descreteVars_Ordinal
        self.descreteVars_Nominal = descreteVars_Nominal
        
        fileName = dict_address
        f = open(fileName,'r')
        self.conversion_dict = json.loads(f.read())
        

    def fit(self, X, y=None):
        # we need the fit statement to accomodate the sklearn pipeline
        des_nom_DF = X[self.descreteVars_Nominal]
        # Map Nominal Categorical data to Numerical
        cat_nom_DF = des_nom_DF.fillna('NULL').astype(str)
        self.ce_binary = ce.BinaryEncoder()
        self.ce_binary.fit(cat_nom_DF)
        return self

    def transform(self, X):
        totalDF = X.copy()
        conDF = X[self.continuesVars]
        des_ord_DF = X[self.descreteVars_Ordinal]
        des_nom_DF = X[self.descreteVars_Nominal]
        
        # Map Ordinal Categorical data to Numerical
        cat_ord_DF_numerical = des_ord_DF.copy()
        for feature in self.conversion_dict:
            temp_dict = self.conversion_dict[feature]
            if ('NA' in temp_dict): # Replace 'NA' with np.nan
                temp_dict[np.nan] = temp_dict.pop('NA')
            cat_ord_DF_numerical[feature] = des_ord_DF[feature].map(temp_dict)
            
        totalDF[self.descreteVars_Ordinal] = cat_ord_DF_numerical
            
        # Map Nominal Categorical data to Numerical
        cat_nom_DF = des_nom_DF.fillna('NULL').astype(str)
        cat_nom_DF_numerical = self.ce_binary.transform(cat_nom_DF)
        totalDF = pd.concat([totalDF, cat_nom_DF_numerical], axis=1)
        totalDF.drop(self.descreteVars_Nominal, axis=1, inplace=True)
        cols_to_drop = [x for x in list(cat_nom_DF_numerical) if ('_0' in x)]
        totalDF.drop(cols_to_drop, axis=1, inplace=True)

        return totalDF


### Clean Numeric Data

In [3]:
class CleanNum(BaseEstimator, TransformerMixin):

    def __init__(self, measure='mean', drop_thresh=0.8):
        self.measure = measure
        self.drop_thresh = drop_thresh

    def fit(self, X, y=None):
        train_df = X.copy().astype(float)
        # Remove columns with dominance bigger than a threshold
        self.drop_list = []
        for feature in train_df:
            col_df = train_df[feature]
            count_nan = col_df.isnull().sum()
            nan_ratio = count_nan/len(col_df)   
            repeats = train_df.pivot_table(index=[feature], aggfunc='size').sort_values()
            max_repeat_ratio = repeats.max()/len(col_df)
            if (nan_ratio>self.drop_thresh or max_repeat_ratio>self.drop_thresh):
                self.drop_list.append(feature)

        # Replace null values with average (or mode) of the train columns
        self.cols_average = train_df.mean(axis = 0)
        self.cols_mode = train_df.mode(axis = 0)
        
        return self

    def transform(self, X):
        totalDF = X.copy().astype(float)
        
        # Replace null values with average (or mode) of the train columns
        for col in totalDF:
            if (self.measure=='mean'):
                totalDF[col] = totalDF[col].fillna(self.cols_average[col])
            if (self.measure=='mode'):
                totalDF[col] = totalDF[col].fillna(self.cols_mode[col])
        
        # Remove columns with dominance bigger than a threshold
        totalDF.drop(self.drop_list, axis=1, inplace=True)

        return totalDF

### Outlier Detection

In [4]:
class OutlierDetection(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass

        # Function to Detection Outlier on one-dimentional datasets.
    def find_anomalies(self,data):
        anomalies_idx = []
        # Set upper and lower limit to 3 standard deviation
        data_std = data.std()
        data_mean = data.mean()
        anomaly_cut_off = data_std * 3

        lower_limit  = data_mean - anomaly_cut_off 
        upper_limit = data_mean + anomaly_cut_off
        # Generate outliers
        for idx in range(len(data)):
            outlier = data[idx]
            if outlier > upper_limit or outlier < lower_limit:
                anomalies_idx.append(idx)
        return anomalies_idx

    def fit(self, X, y=None):
        train_df = X.copy()
        outliers_full_list = []
        for feature in train_df:
            outliers_full_list = outliers_full_list + self.find_anomalies(train_df[feature].values)

        self.outliers_unique = list(set(int(outliers_full_list)))
        return self

    def transform(self, X):
        totalDF = X.copy()
        totalDF.drop(self.outliers_unique, inplace=True)
        return totalDF

### Data Transformation (Box-Cox transformation)

In [5]:
from scipy.special import boxcox1p
from scipy.stats import norm, skew

class DataTransformation(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass
    
    def fit(self, X, y=None, thresh=0.75, lam=0.15):
        self.lam = lam
        train_df = X.copy()
        # Check the skew of all numerical features
#         print(type(skew(train_df)))
        skewed_feats = train_df.apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
#         print(type(skewed_feats ))
        skewness = pd.DataFrame({'Skew' :skewed_feats})
        skewness = skewness[abs(skewness) > thresh]
        self.skewed_features_idx_list = skewness.index
        return self
    
    def transform(self, X):
        totalDF = X.copy()
#         skewed_features = totalDF[self.skewed_features_list]
        for feat_idx in self.skewed_features_idx_list:
            #all_data[feat] += 1
            totalDF[feat_idx] = boxcox1p(totalDF[feat_idx], self.lam)
        return totalDF

### Data Normalization
There is no need to define a specific class for data normalization.

### Dimensionality Reduction
I am thinking there is no need for dimensionality reduction at the moment. But I may change my mind later...

----

In [6]:
n_folds = 5

def rmsle_cv(model, train, y_train):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

------

## Training steps - models

In [7]:
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_squared_error

In [8]:
train_file = 'house-prices-advanced-regression-techniques/train.csv'
train_df = pd.read_csv(train_file)
test_file = 'house-prices-advanced-regression-techniques/test.csv'
test_df = pd.read_csv(test_file)





descreteVars_Nominal = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LandContour', 'LotConfig', 'Neighborhood',
                        'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
                        'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'CentralAir', 'Electrical',
                        'GarageType', 'MiscFeature', 'SaleType', 'SaleCondition']

descreteVars_Ordinal = ['LotShape', 'Utilities', 'LandSlope', 'ExterQual', 'ExterCond', 'BsmtQual',
                        'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC',
                        'KitchenQual','Functional',  'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond',
                        'PavedDrive', 'PoolQC', 'Fence']

resultVar = ['SalePrice']



continuesVars = list(set(list(train_df)) - set(descreteVars_Nominal) - set(descreteVars_Ordinal) - set(resultVar))

dict_address='cat_ord_dict.txt'


X = train_df.drop(resultVar, axis=1)
y = train_df[resultVar]


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [9]:
models_names = ['KernelRidge', 'ElasticNet', 'Lasso', 'GradientBoostingRegressor', 'BayesianRidge', 'LassoLarsIC', 'RandomForestRegressor', 'XGBRegressor']
models_list = [KernelRidge(), ElasticNet(), Lasso(), GradientBoostingRegressor(), BayesianRidge(), LassoLarsIC(), RandomForestRegressor(), XGBRegressor()]

KR_param_grid = {'alpha': 0.1, 'coef0': 100, 'degree': 1, 'gamma': None, 'kernel': 'polynomial'}
EN_param_grid = {'alpha': 0.001, 'copy_X': True, 'l1_ratio': 0.6, 'fit_intercept': True, 'normalize': False, 
                         'precompute': False, 'max_iter': 300, 'tol': 0.001, 'selection': 'random', 'random_state': None}
LASS_param_grid = {'alpha': 0.0005, 'copy_X': True, 'fit_intercept': True, 'normalize': False, 'precompute': False, 
                    'max_iter': 300, 'tol': 0.01, 'selection': 'random', 'random_state': None}
GB_param_grid = {'loss': 'huber', 'learning_rate': 0.1, 'n_estimators': 300, 'max_depth': 3, 
                                        'min_samples_split': 0.0025, 'min_samples_leaf': 5}
BR_param_grid = {'n_iter': 200, 'tol': 0.00001, 'alpha_1': 0.00000001, 'alpha_2': 0.000005, 'lambda_1': 0.000005, 
                 'lambda_2': 0.00000001, 'copy_X': True}
LL_param_grid = {'criterion': 'aic', 'normalize': True, 'max_iter': 100, 'copy_X': True, 'precompute': 'auto', 'eps': 0.000001}
RFR_param_grid = {'n_estimators': 50, 'max_features': 'auto', 'max_depth': None, 'min_samples_split': 5, 'min_samples_leaf': 2}
XGB_param_grid = {'max_depth': 3, 'learning_rate': 0.1, 'n_estimators': 300, 'booster': 'gbtree', 'gamma': 0, 'reg_alpha': 0.1,
                  'reg_lambda': 0.7, 'max_delta_step': 0, 'min_child_weight': 1, 'colsample_bytree': 0.5, 'colsample_bylevel': 0.2,
                  'scale_pos_weight': 1}
params_grid = [KR_param_grid, EN_param_grid, LASS_param_grid, GB_param_grid, BR_param_grid, LL_param_grid, RFR_param_grid, XGB_param_grid]

In [10]:
def cal_rmse(model, X, y):
    return (np.sqrt(mean_squared_error(y, model.predict(X))))

In [11]:
original_test_file = 'house-prices-advanced-regression-techniques/test.csv'
original_test_df = pd.read_csv(original_test_file)
id_col = original_test_df['Id']

for i in range(len(models_list)):
    print(models_names[i])
    model = models_list[i]
    params = params_grid[i]
    print(params)
    pipeline = make_pipeline(CatToNum(dict_address, continuesVars, descreteVars_Ordinal, descreteVars_Nominal), CleanNum(), RobustScaler(), model.set_params(**params))
    pipeline_transformed = make_pipeline(CatToNum(dict_address, continuesVars, descreteVars_Ordinal, descreteVars_Nominal), CleanNum(), DataTransformation(), RobustScaler(), model.set_params(**params))

    pipeline.fit(X_train, y_train)    
    pipeline_transformed.fit(X_train, y_train)    
    
    print('Normal ' + models_names[i])
    print('Train rmse: ')
    print(cal_rmse(pipeline, X_train, y_train))
    print('Validation rmse: ')
    print(cal_rmse(pipeline, X_val, y_val))
    print(models_names[i]+' with transformed skewed columns')
    print('Train rmse: ')
    print(cal_rmse(pipeline_transformed, X_train, y_train))
    print('Validation rmse: ')
    print(cal_rmse(pipeline_transformed, X_val, y_val))
    print('-----------------------------------------------------')
    
    Y_pred = pipeline.predict(test_df)
    result_df = pd.concat([id_col, pd.DataFrame(Y_pred, columns=['SalePrice'])], axis=1)
    result_df.to_csv('Results/'+models_names[i]+'.csv', index=False)
    
    
    Y_pred_transformed = pipeline_transformed.predict(test_df)
    result_df = pd.concat([id_col, pd.DataFrame(Y_pred_transformed, columns=['SalePrice'])], axis=1)
    result_df.to_csv('Results/'+models_names[i]+'_transformed.csv', index=False)
    
    


KernelRidge
{'alpha': 0.1, 'coef0': 100, 'degree': 1, 'gamma': None, 'kernel': 'polynomial'}
Normal KernelRidge
Train rmse: 
40477.34158383138
Validation rmse: 
33752.360766751575
KernelRidge with transformed skewed columns
Train rmse: 
33964.60910631942
Validation rmse: 
35648.871174359774
-----------------------------------------------------
ElasticNet
{'alpha': 0.001, 'copy_X': True, 'l1_ratio': 0.6, 'fit_intercept': True, 'normalize': False, 'precompute': False, 'max_iter': 300, 'tol': 0.001, 'selection': 'random', 'random_state': None}


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Normal ElasticNet
Train rmse: 
41325.30920639971
Validation rmse: 
34810.30493615326
ElasticNet with transformed skewed columns
Train rmse: 
33899.27720857602
Validation rmse: 
35853.99645104794
-----------------------------------------------------
Lasso
{'alpha': 0.0005, 'copy_X': True, 'fit_intercept': True, 'normalize': False, 'precompute': False, 'max_iter': 300, 'tol': 0.01, 'selection': 'random', 'random_state': None}


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Normal Lasso
Train rmse: 
41406.27524995679
Validation rmse: 
34917.889532243666
Lasso with transformed skewed columns
Train rmse: 
33898.335726322875
Validation rmse: 
35887.114249230624
-----------------------------------------------------
GradientBoostingRegressor
{'loss': 'huber', 'learning_rate': 0.1, 'n_estimators': 300, 'max_depth': 3, 'min_samples_split': 0.0025, 'min_samples_leaf': 5}


...

Normal BayesianRidge
Train rmse: 
39673.49024145989
Validation rmse: 
32803.34106049047
BayesianRidge with transformed skewed columns
Train rmse: 
34136.259176299565
Validation rmse: 
35605.25461046637
-----------------------------------------------------
LassoLarsIC
{'criterion': 'aic', 'normalize': True, 'max_iter': 100, 'copy_X': True, 'precompute': 'auto', 'eps': 1e-06}


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Normal LassoLarsIC
Train rmse: 
39190.286795731154
Validation rmse: 
33867.8805962962
LassoLarsIC with transformed skewed columns
Train rmse: 
36237.789598564945
Validation rmse: 
37736.43770937154
-----------------------------------------------------
RandomForestRegressor
{'n_estimators': 50, 'max_features': 'auto', 'max_depth': None, 'min_samples_split': 5, 'min_samples_leaf': 2}


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\pipeline.py:267: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\pipeline.py:267: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)


Normal RandomForestRegressor
Train rmse: 
38389.95616432039
Validation rmse: 
39691.808804357446
RandomForestRegressor with transformed skewed columns
Train rmse: 
14968.822865022277
Validation rmse: 
26673.488684543707
-----------------------------------------------------
XGBRegressor
{'max_depth': 3, 'learning_rate': 0.1, 'n_estimators': 300, 'booster': 'gbtree', 'gamma': 0, 'reg_alpha': 0.1, 'reg_lambda': 0.7, 'max_delta_step': 0, 'min_child_weight': 1, 'colsample_bytree': 0.5, 'colsample_bylevel': 0.2, 'scale_pos_weight': 1}
[01:24:26] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:24:28] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Normal XGBRegressor
Train rmse: 
39965.45639953617
Validation rmse: 
42260.85232911142
XGBRegressor with transformed skewed columns
Train